# Project Astra: Portfolio Optimization Experiments

This notebook demonstrates the end-to-end pipeline for portfolio optimization using RL and classical methods.

Focus: NIFTY Bank portfolio with daily rebalancing.

## Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import logging

from astra.data_pipeline.downloader import PortfolioDataDownloader
from astra.data_pipeline.preprocessor import PortfolioDataPreprocessor
from astra.data_pipeline.data_manager import PortfolioDataManager
from astra.rl_framework.environment import PortfolioEnvironment
from astra.rl_framework.trainer import PortfolioTrainer
from astra.evaluation.optimizer import ClassicalPortfolioOptimizer, PortfolioBacktester

# Set up logging
logging.basicConfig(level=logging.INFO)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## Configuration

In [ ]:
# Load config
project_root = Path().resolve()
config_candidates = [
    project_root / 'config/portfolio.yaml',
    project_root.parent / 'config/portfolio.yaml'
]
for candidate in config_candidates:
    if candidate.exists():
        CONFIG_PATH = candidate
        break
else:
    raise FileNotFoundError(f"Config file not found. Tried: {config_candidates}")

with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

print("Portfolio Configuration:")
print(f"Assets: {config['portfolio']['assets']}")
print(f"Initial weights mode: {config['portfolio']['initial_weights']['mode']}")
print(f"Cash buffer: {config['portfolio']['initial_weights']['cash_buffer']}")

## Data Pipeline

### Download Data

In [ ]:
# Download NIFTY Bank data
downloader = PortfolioDataDownloader(config_path=CONFIG_PATH)
portfolio_data = downloader.download_portfolio_data()
print(f"Downloaded data shape: {portfolio_data.shape}")
print(f"Date range: {portfolio_data.index.min()} to {portfolio_data.index.max()}")

# Save raw data
downloader.save_data(portfolio_data)

### Preprocessing

In [ ]:
# Process data
preprocessor = PortfolioDataPreprocessor(config_path=CONFIG_PATH)
processed_data = preprocessor.process_all()
print(f"Processed data shape: {processed_data.shape}")
print(f"Features added: {[col for col in processed_data.columns if not col.endswith('_close')]}")

# Show sample
display(processed_data.head())

### Initialize Portfolio

In [ ]:
# Data manager to handle initialization
manager = PortfolioDataManager(config_path=CONFIG_PATH)
data, initial_state = manager.process_and_initialize()

print(f"Initial portfolio value: ₹{initial_state['portfolio_value']:,.2f}")
print(f"Initial weights:")
for asset, weight in initial_state['weights'].items():
    if asset != 'cash':
        print(f"  {asset}: {weight:.3f}")
print(f"Cash: {initial_state['weights']['cash']:.3f}")

## Exploratory Data Analysis

In [ ]:
# Price evolution
price_cols = [col for col in processed_data.columns if '_close' in col]
processed_data[price_cols].plot(figsize=(12, 6), title='NIFTY Bank Asset Prices')
plt.ylabel('Price (₹)')
plt.show()

# Returns distribution
return_cols = [col for col in processed_data.columns if '_returns' in col and not col.startswith('portfolio')]
processed_data[return_cols].plot.hist(bins=50, alpha=0.7, figsize=(10, 6), title='Daily Returns Distribution')
plt.xlabel('Daily Return')
plt.show()

# Correlation heatmap
returns_df = processed_data[return_cols].rename(columns={col: col.replace('_returns', '') for col in return_cols})
correlation = returns_df.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0)
plt.title('Asset Returns Correlation')
plt.show()

# Volatility
vol_cols = [col for col in processed_data.columns if '_volatility' in col]
if vol_cols:
    processed_data[vol_cols].plot(figsize=(10, 6), title='Rolling 30-Day Volatility')
    plt.ylabel('Volatility')
    plt.show()

## Classical Portfolio Optimization

In [ ]:
# Prepare returns data for optimization
returns_cols = [f"{asset}_returns" for asset in config['portfolio']['assets']]
returns_clean = processed_data[returns_cols].dropna()

# Classical optimization
optimizer = ClassicalPortfolioOptimizer(returns_clean)
classical_portfolios = optimizer.get_all_portfolios()

print("Classical Portfolio Results:")
for name, portfolio in classical_portfolios.items():
    print(f"\n{name.upper()}:")
    print(f"  Expected Return: {portfolio['expected_return']:.4f}")
    print(f"  Expected Volatility: {portfolio['expected_volatility']:.4f}")
    print(f"  Sharpe Ratio: {portfolio['sharpe_ratio']:.4f}")
    print(f"  Weights: {dict(zip(config['portfolio']['assets'], portfolio['weights']))}")

# Visualize weights
weights_df = pd.DataFrame({name: portfolio['weights'] for name, portfolio in classical_portfolios.items()},
                         index=config['portfolio']['assets'])
weights_df.plot(kind='bar', figsize=(12, 6), title='Classical Portfolio Weights')
plt.ylabel('Weight')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Backtest Classical Portfolios

In [ ]:
# Backtest all classical portfolios
backtester = PortfolioBacktester(processed_data)
backtest_results = {}

for name, portfolio in classical_portfolios.items():
    result = backtester.backtest_portfolio(portfolio['weights'])
    backtest_results[name] = result
    print(f"\n{name.upper()} Backtest:")
    print(f"  Final Value: ₹{result['final_value']:,.2f}")
    print(f"  Total Return: {result['total_return']:.4f}")
    print(f"  Sharpe Ratio: {result['sharpe_ratio']:.4f}")
    print(f"  Max Drawdown: {result['max_drawdown']:.4f}")

# Plot portfolio values
plt.figure(figsize=(14, 8))
for name, result in backtest_results.items():
    plt.plot(result['portfolio_values'], label=name.replace('_', ' ').title())

plt.title('Classical Portfolios - Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value (₹)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## RL Training (Base Infrastructure)

In [ ]:
# Initialize RL environment and trainer
trainer = PortfolioTrainer(config_path=CONFIG_PATH)

# Quick training demo
# Note: This may take time and resources
# trainer.train(total_timesteps=5000)

# For demo, just test environment
env = trainer.env
env = trainer.env
obs, _ = env.reset()

print("Testing RL Environment...")
for step in range(10):  # Short demo
    action = env.action_space.sample()  # Random action
    obs, reward, done, info = env.step(action)
    obs, reward, terminated, truncated, info = env.step(action)
    
    if step % 5 == 0:
        print(f"Step {step}: Portfolio Value = ₹{info['portfolio_value']:,.2f}, Reward = {reward:.6f}")
    
    if done:
    if terminated or truncated:

print(f"Demo completed. Total reward over 10 steps: {total_reward:.6f}")
print(f"Final portfolio value: ₹{info['portfolio_value']:,.2f}")

## Comparison and Summary

In [ ]:
# Compare performance
comparison_data = []

for name, result in backtest_results.items():
    comparison_data.append({
        'Strategy': name.replace('_', ' ').title(),
        'Type': 'Classical',
        'Final_Value': result['final_value'],
        'Total_Return': result['total_return'],
        'Sharpe_Ratio': result['sharpe_ratio'],
        'Max_Drawdown': result['max_drawdown']
    })

# RL placeholder (when trained)
# comparison_data.append({
#     'Strategy': 'RL SAC',
#     'Type': 'RL',
#     'Final_Value': rl_final_value,
#     'Total_Return': rl_total_return,
#     'Sharpe_Ratio': rl_sharpe,
#     'Max_Drawdown': rl_max_dd
# })

# Create comparison table
comparison_df = pd.DataFrame(comparison_data)
display(comparison_df.round(4))

# Visualize key metrics
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Sharpe Ratio
axes[0,0].bar(comparison_df['Strategy'], comparison_df['Sharpe_Ratio'])
axes[0,0].set_title('Sharpe Ratio')
axes[0,0].tick_params(axis='x', rotation=45)

# Total Return
axes[0,1].bar(comparison_df['Strategy'], comparison_df['Total_Return'])
axes[0,1].set_title('Total Return')
axes[0,1].tick_params(axis='x', rotation=45)

# Max Drawdown
axes[1,0].bar(comparison_df['Strategy'], comparison_df['Max_Drawdown'])
axes[1,0].set_title('Max Drawdown')
axes[1,0].tick_params(axis='x', rotation=45)

# Final Value
axes[1,1].bar(comparison_df['Strategy'], comparison_df['Final_Value'])
axes[1,1].set_title('Final Portfolio Value (₹)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("EXPERIMENT SUMMARY")
print("="*60)
print(f"Portfolio: {', '.join(config['portfolio']['assets'])}")
print(f"Data Period: {processed_data.index.min()} to {processed_data.index.max()}")
print(f"Trading Days: {len(processed_data)}")
print()
print("Classical optimization successfully implemented.")
print("RL environment ready for training.")
print("Next steps: Train RL agent and compare with classical methods.")
print("="*60)